In [ ]:
#!/usr/bin/env python
# encoding: utf-8

"""
@author: Wayne
@contact: wangye.hope@gmail.com / wang121ye@hotmail.com
@software: PyCharm
@file: main
@time: 2020/09/22 00:14
"""

import os
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn.preprocessing import FunctionTransformer,Imputer
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [ ]:
def load_data(name='test'):
    path = os.path.join('../data', name + '.csv')
    data = pd.read_csv(path)
    return data

In [ ]:
train_data = load_data('train')
test_data = load_data('test')
print(train_data.head(4))
print(test_data.head(4))

In [ ]:
x = train_data['Cabin']
# print(x.describe())
# print(train_data.head(4))
# print(train_data.columns)
test_data.head(4)

In [ ]:
pandas_profiling.ProfileReport(train_data)